# BaiscTick: Create Cluster: RDB
This notebook will create (start) an RDB cluster on a named database.

In [1]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env_kdb_1 import *

# Cluster names and database
from basictick_setup import *

SEC_THREADS='4'

# Cluster Settings

CODEBASE="basictick"
S3_CODE_PATH="code"

CAPACITY_CONFIG={ 'nodeCount': 1, 'nodeType': 'kx.s.xlarge'}
SAVEDOWN_CONFIG={ 'type': 'SDS01', 'size':400 }
DATABASE_CONFIG=[{ 'databaseName': DB_NAME }]
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }
INIT_SCRIPT=f'{CODEBASE}/rdbmkdb.q'

CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'dbname', 'value': DB_NAME}, 
    { 'key': 'codebase', 'value': CODEBASE}, 
#    { 'key': 'tphostfile', 'value': TP_HOSTFILE}, 
    {'key': 'tp', 'value': TP }, 
]

In [3]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


## Check Database

In [4]:
note_str = ""

try:
    c_set_list = client.list_kx_changesets(environmentId=ENV_ID, databaseName=DB_NAME)['kxChangesets']
except:
    note_str = "<<Could not get changesets>>"

print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"Changeset ({c['status']}): {c_set_id}: Created: {c['createdTimestamp']}")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

Database: basictickdb, Changesets: 8 
Changeset (COMPLETED): EsR51anc510WEVVhRI5bDA: Created: 2023-06-25 17:47:51.610000+00:00


changeType,s3Path,dbPath
PUT,s3://kdb-demo-829845998889-kms/data/hdb/,/


Changeset (COMPLETED): fMR6ApFDdL78q6i5NmYUYg: Created: 2023-06-25 19:25:57.256000+00:00


changeType,s3Path,dbPath
PUT,s3://finspace-landing-us-east-1-jlcenjvtkgzrdek2qqv7ic/4LNr71RzmdTKeT6bmKhM-/0/,/2023.06.25/
PUT,s3://finspace-landing-us-east-1-jlcenjvtkgzrdek2qqv7ic/4LNr71RzmdTKeT6bmKhM-/1/,/


Changeset (COMPLETED): 6sR8rr6sRCREMv6skwufew: Created: 2023-06-26 20:20:33.754000+00:00


...

changeType,s3Path,dbPath
PUT,s3://finspace-landing-us-east-1-jlcenjvtkgzrdek2qqv7ic/DdWRdLRbWVA50UULo9m9I/0/,/2023.07.07/
PUT,s3://finspace-landing-us-east-1-jlcenjvtkgzrdek2qqv7ic/DdWRdLRbWVA50UULo9m9I/1/,/


Changeset (COMPLETED): rMSjhHfz3uujvXmT5Op27w: Created: 2023-07-11 22:18:35.368000+00:00


changeType,s3Path,dbPath
PUT,s3://finspace-landing-us-east-1-jlcenjvtkgzrdek2qqv7ic/9M76UNXKKyJeO5nV2lnOI/0/,/2023.07.11/
PUT,s3://finspace-landing-us-east-1-jlcenjvtkgzrdek2qqv7ic/9M76UNXKKyJeO5nV2lnOI/1/,/


Changeset (COMPLETED): rMSj0dLkasmJ2BPKY6Ycuw: Created: 2023-07-12 01:07:34.473000+00:00


changeType,s3Path,dbPath
PUT,s3://finspace-landing-us-east-1-jlcenjvtkgzrdek2qqv7ic/fm4yV4MG2jxChQRrPe3ta/0/,/2023.07.12/
PUT,s3://finspace-landing-us-east-1-jlcenjvtkgzrdek2qqv7ic/fm4yV4MG2jxChQRrPe3ta/1/,/


# Deploy code

In [5]:
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
else:
    cp = f"""
zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
    
# execute the S3 copy
os.system(cp)

updating: basictick/ (stored 0%)
updating: basictick/connectmkdb.q (deflated 63%)
updating: basictick/example.schema.q (deflated 12%)
updating: basictick/feedmkdb.q (deflated 53%)
updating: basictick/funcDownHandle.q (deflated 33%)
updating: basictick/gwmkdbcluster.q (deflated 61%)
updating: basictick/hdbmkdb.q (deflated 52%)
updating: basictick/loadDep.q (deflated 57%)
updating: basictick/query.q (deflated 19%)
updating: basictick/rdbmkdb.q (deflated 57%)
updating: basictick/tickerplant2 (stored 0%)
updating: basictick/tickerplant3 (stored 0%)
updating: basictick/tp.q (deflated 52%)
updating: basictick/.ipynb_checkpoints/ (stored 0%)
updating: basictick/.ipynb_checkpoints/tickerplant3-checkpoint (stored 0%)
updating: basictick/.ipynb_checkpoints/tickerplant-checkpoint (stored 0%)
updating: basictick/tickerplant (stored 0%)
updating: basictick/.ipynb_checkpoints/rdbmkdb-checkpoint.q (deflated 57%)
updating: basictick/.ipynb_checkpoints/gwmkdbcluster-checkpoint.q (deflated 61%)
  adding

0

## Create Cluster

In [6]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=RDB_CLUSTER_NAME,
    clusterType='RDB',
    releaseLabel = '1.0',
    databases=DATABASE_CONFIG,
    capacityConfiguration=CAPACITY_CONFIG,
    savedownStorageConfiguration=SAVEDOWN_CONFIG,
    clusterDescription="Created with create_RDB notebook",
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    commandLineArguments=CMD_ARGS,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [7]:
resp

{'ResponseMetadata': {'RequestId': 'faf55ef5-a76a-46af-93b8-bc812e84bd14',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1114',
   'connection': 'keep-alive',
   'date': 'Tue, 25 Jul 2023 22:07:08 GMT',
   'x-amzn-requestid': 'faf55ef5-a76a-46af-93b8-bc812e84bd14',
   'x-amz-apigw-id': 'IpAdFELMIAMFXRQ=',
   'x-amzn-trace-id': 'Root=1-64c04786-0adaa03056e8ce626fb6e33c',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 0003b3450f3f9fac44312c4622a410c2.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'sClLAEerj8JzvnYh8DtXsF6mpzVZdOzqnEB9HXZQ6qKZYiiPHenUFA=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'RDB_basictickdb',
 'clusterType': 'RDB',
 'databases': [{'databaseName': 'basictickdb',
   'cacheConfigurations': [],
   'changesetId': 'rMSj0dLkasmJ2BPKY6Ycuw'}],
 'clusterDescription': 'Created with create_RDB notebook',
 'capacityConfiguration': {'nodeType': 'kx.s.xlarge', 'nodeCo

In [8]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=RDB_CLUSTER_NAME)
print("** DONE **")

** DONE **


In [9]:
cdf = get_clusters(client, environmentId=ENV_ID, clusterType="RDB")

display(cdf)

,clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
0,RDB_basictickdb,RUNNING,RDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'basictickdb'}, {'key': 'codebase', 'value': 'basictick'}, {'key': 'tp', 'value': ':172.31.22.143:5000'}]",Created with create_RDB notebook,2023-07-25 22:19:43.754000+00:00,2023-07-25 22:07:08.378000+00:00,basictickdb,[]
1,RDB_taqdb,RUNNING,RDB,"{'nodeType': 'kx.s.8xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '2'}, {'key': 'dbname', 'value': 'taqdb'}, {'key': 'codebase', 'value': 'code'}]",Created with notebook,2023-07-14 13:17:52.782000+00:00,2023-07-14 13:04:48.337000+00:00,taqdb,[]
2,RDB_totaldb,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '2'}, {'key': 'dbname', 'value': 'totaldb'}, {'key': 'codebase', 'value': 'code'}]",Created with notebook,2023-07-13 13:42:53.222000+00:00,2023-07-13 13:29:49.511000+00:00,totaldb,[]


In [10]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-07-25 22:19:52.044159
